In [4]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema.document import Document
from langchain.document_loaders import TextLoader
import os
from dotenv import load_dotenv

In [5]:
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [6]:
# 1. Prepare and split documents
loader = TextLoader("data.txt")  # <- replace with your file
document = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
split_docs = splitter.split_documents(document)


In [19]:
# 2. Embed and persist vector store
persist_dir = "./db"
db = Chroma.from_documents(split_docs, embedding=OpenAIEmbeddings(), persist_directory=persist_dir)
db.persist()
retriever = db.as_retriever(search_kwargs={"k": 6})

In [28]:

# 3. Setup LLMs and prompts
llm = ChatOpenAI(temperature=0)

tag_prompt = ChatPromptTemplate.from_template("""
    Extract structured metadata as JSON with fields like "company" and "year" from the question below:\n\n{query}"""
)

qa_prompt = ChatPromptTemplate.from_template(
    """Use only the context below to answer the question.
If the answer is not directly in the context, use reasoning to infer a likely but informed answer."

Context:
{context}

Question: {input}
"""
)

In [29]:
def ask_question(query: str) -> str:
    # Tag extraction
    tag_response = llm(tag_prompt.format_prompt(query=query).to_messages())
    tags_text = tag_response.content.strip()

    # You can parse tags_text as JSON if you want structured tags
    print("Extracted tags:", tags_text)

    # Retrieve docs
    relevant_docs = retriever.get_relevant_documents(query)
    combined_context = "\n".join(doc.page_content for doc in relevant_docs)

    # QA
    qa_response = llm(qa_prompt.format_prompt(input=query, context=combined_context).to_messages())

    print("Answer:", qa_response.content)
    return qa_response.content

if __name__ == "__main__":
    question = "what is tesla planning in mid of 2024"
    answer = ask_question(question)
    print("Final answer:", answer)

Extracted tags: {
    "company": "Tesla",
    "year": "2024"
}
Answer: Based on the context provided, Tesla is likely planning to continue with multiple product launches, software upgrades, and global manufacturing expansion in the mid of 2024.
Final answer: Based on the context provided, Tesla is likely planning to continue with multiple product launches, software upgrades, and global manufacturing expansion in the mid of 2024.
